# Скрипт чтобы забить БД изначальными данными. Использовал его с хоста. В репе БД уже забита ими, так что можно его не запускать

In [2]:
# Import required libraries
import os
import psycopg2
import pandas as pd

# PostgreSQL connection settings
POSTGRES_USER = "myuser"
POSTGRES_PASSWORD = "mypassword"
POSTGRES_DB = "mydatabase"
POSTGRES_HOST = "localhost"
POSTGRES_PORT = 5432

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname=POSTGRES_DB,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD,
    host=POSTGRES_HOST,
    port=POSTGRES_PORT,
)
cursor = conn.cursor()

# Function to execute SQL scripts
def execute_sql_script(file_path):
    with open(file_path, "r") as file:
        sql = file.read()
        cursor.execute(sql)
        print(f"Executed: {file_path}")
    conn.commit()

# Step 1: Create schemas and tables
table_scripts_path = "../table_scripts"
for script in sorted(os.listdir(table_scripts_path)):
    if script.endswith(".sql"):
        execute_sql_script(os.path.join(table_scripts_path, script))

# Step 2: Load CSV data into tables
source_data_path = "../source_data"
csv_to_table_mapping = {
    "complete_craft_market_wide_20230730.csv": "source1.craft_market_wide",
    "complete_craft_market_masters_products_202305071535.csv": "source2.craft_market_masters_products",
    "complete_craft_market_orders_customers_202305071535.csv": "source2.craft_market_orders_customers",
    "complete_craft_market_craftsmans_202305071534.csv": "source3.craft_market_craftsmans",
    "complete_craft_market_customers_202305071535.csv": "source3.craft_market_customers",
    "complete_craft_market_orders_202305071535.csv": "source3.craft_market_orders",
}

for csv_file, table_name in csv_to_table_mapping.items():
    csv_path = os.path.join(source_data_path, csv_file)
    with open(csv_path, "r") as f:
        cursor.copy_expert(
            f"COPY {table_name} FROM STDIN WITH CSV HEADER DELIMITER ','",
            f,
        )
        print(f"Loaded data from {csv_file} into {table_name}")
    conn.commit()

# Close connection
cursor.close()
conn.close()

Executed: ../table_scripts/DDL_craft_market_source_1.sql
Executed: ../table_scripts/DDL_craft_market_source_2.sql
Executed: ../table_scripts/DDL_craft_market_source_3.sql
Loaded data from complete_craft_market_wide_20230730.csv into source1.craft_market_wide
Loaded data from complete_craft_market_masters_products_202305071535.csv into source2.craft_market_masters_products
Loaded data from complete_craft_market_orders_customers_202305071535.csv into source2.craft_market_orders_customers
Loaded data from complete_craft_market_craftsmans_202305071534.csv into source3.craft_market_craftsmans
Loaded data from complete_craft_market_customers_202305071535.csv into source3.craft_market_customers
Loaded data from complete_craft_market_orders_202305071535.csv into source3.craft_market_orders


In [3]:
conn = psycopg2.connect(
    dbname=POSTGRES_DB,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD,
    host=POSTGRES_HOST,
    port=POSTGRES_PORT,
)
cursor = conn.cursor()

dwh_scripts_path = "../dwh_scripts"
scripts_to_run = [
    "DDL_craft_market_dwh.sql",
    "DDL_craft_market_datamart_increment.sql"
]

for script in scripts_to_run:
    script_path = os.path.join(dwh_scripts_path, script)
    execute_sql_script(script_path)

print("All DWH and Data Mart tables have been created successfully!")

Executed: ../dwh_scripts/DDL_craft_market_dwh.sql
Executed: ../dwh_scripts/DDL_craft_market_datamart_increment.sql
All DWH and Data Mart tables have been created successfully!
